In [1]:
import numpy as np
import math

In [2]:
fluid_model = input('Enter fluid model: ')
print('The fluid model is : {}' .format(fluid_model))
print ('Use relevant models for black oil or compositional model to compute velocities')

Enter fluid model: BlackOil
The fluid model is : BlackOil
Use relevant models for black oil or compositional model to compute velocities


In [3]:
def Bo_vel (Qosc, Bo, Rso, Qgsc, Bg, d, Qwsc = 0, Bw = 1, Rsw = 0):
    Area = (math.pi/576) * d**2
    
    qo = 6.4988*10**(-5) * Qosc * Bo
    qg = 1.1574*10**(-5) * (Qgsc - Qosc*Rso -Qwsc*Rsw)*Bg
    qw = 6.4988*10**(-5) * Qwsc * Bw
    ql = qo + qw
        
    Vsl = (0.3048*ql)/Area
    Vsg = 0.3048*qg/Area
    Vm = Vsl + Vsg
    
    return Vsl, Vsg, Vm

# Liquid flow rates are in STB/D, liquid formation volume factor are in bbl/STB, gas flow rate in SCF/D, gas formation volume 
# factor in cubic feet/SCF, gas solubility in the liquid phase is in bbl/SCF, d is in inches and area is in square feet 
# The estimated velocities are in m/s

In [4]:
def Comp_vel (wt, den_l, den_g, xg, d):
    Area = (math.pi/576) * d**2
    
    ql = 1.8542 * 10**(-4) * (wt * (1- xg))/den_l 
    qg = 1.8542 * 10**(-4) *(wt * xg)/den_g
        
    Vsl = (0.3048*ql)/Area
    Vsg = 0.3048*qg/Area
    Vm = Vsl + Vsg
    
    return Vsl, Vsg, Vm

# den_l and den_g are the liquid and gas densities, and bother are in Kg/cubic meters.
# wt is the pound mass of feed per day, xg is the mass fraction of the vapour phase and d is in inches

In [5]:
Bo_vel(10000, 1.197, 281, 10*10**6, 0.0091, 6 )

(1.2075702214649229, 1.1755438489050583, 2.383114070369981)

In [6]:
Vsl = float(input('Enter the calculated liquid superficial velocity : '))
Vsg = float(input('Enter the calculated gas superficial velocity : '))
Vm = float(input('Enter the calculated mixture : '))

Enter the calculated liquid superficial velocity : 1.20757
Enter the calculated gas superficial velocity : 1.17554
Enter the calculated mixture : 2.383114


In [7]:
den_l = float(input('Enter liquid density in kg/m**3 : '))
den_g = float(input('Enter gas density in kg/m**3 : '))
mhew_l = float(input('Enter liquid viscosity in Pa.S : '))
sur_t = float(input('Enter surface tension in dyne/cm : '))
theta = float(input('Enter tubing inclination : '))
g = float(input('Enter acceleration due to gravity in m/S**2 : '))
e = float(input('Enter surface roughness in ft : '))
d = float(input('Enter tubing diameter in inches : '))

Enter liquid density in kg/m**3 : 761.6
Enter gas density in kg/m**3 : 94.1
Enter liquid viscosity in Pa.S : 0.00097
Enter surface tension in dyne/cm : 0.00841
Enter tubing inclination : 1.5708
Enter acceleration due to gravity in m/S**2 : 9.81
Enter surface roughness in ft : 0.00072
Enter tubing diameter in inches : 6


In [8]:
# We will assume bubble flow as the reference pattern

# Calculating parameters needed to idenfify transition from  bubble flow to any other pattern

Vs = 1.53*((g * sur_t*(den_l-den_g)/den_l**2)**0.25) # slip velocity in m/s

# Bubble flow coefficient
if d < 4.7244 or Vsl> 0.02:
    Co = 1.2
elif d > 4.7244 or Vsl< 0.02:
    Co = 2.0

# Bubble flow condition
if Vsg < (math.sin(theta)/(4-Co))*(Co*Vsl + Vs):
    print('Bubble flow exists')
    fg = Vsg/(Co*Vm + Vs)
    N_rey = (den_l*Vm* (0.0254*d))/mhew_l
    f = (1/(-2*np.log10(2*e/(3.7*d)-(5.02/N_rey)* np.log10((2*e/(3.7*d)) +(13/N_rey)))))**2
    
    pres_grad = den_s * g * math.sin(theta) + (f*Vm**2*den_s)/(2*(0.0254*d))
    
elif Vm**1.12>(4.68* (0.0254*d)**0.48 * (g*(den_l - den_g)/ sur_t)**0.5 * (sur_t/den_l)**0.6 * \
               (den_l/mhew_l)**0.08) and fg<0.52:
    print('Dispersed bubble flow exists')
    fg = Vsg/(Co*Vm + Vs)
    den_s = fg*den_g + (1-fg)*den_l
    N_rey = (den_l*Vm* (0.0254*d))/mhew_l
    f = (1/(-2*np.log10(2*e/(3.7*12*d)-(5.02/N_rey)*  np.log10((2*e/(3.7*12*d)) +(13/N_rey)))))**2
    
    pres_grad = den_s * g * math.sin(theta) + (f*Vm**2*den_s)/(2*(0.0254*d))

    
elif Vsg>(math.sin(theta)/(4-Co))*(Co*Vsl + Vs):
    if (den_l*Vsl**2)<50:
        test = (0.00673*(den_l*Vsl**2)**1.17)
    elif (den_l*Vsl**2)>50:
        test = 17.1*(math.log10(den_l * Vsl**2)) - 23.2
    
    V_tb = 0.35*np.sqrt(g*(0.0254*d)*((den_l-den_g)/den_l))* np.sqrt(math.sin(theta))*(1-math.cos(theta))**1.2 # in m/s
    
    if (den_g * Vsg**2)<test:
        print('Slug flow exists')
        Co = 1.2
        fg = Vsg/(Co*Vm + V_tb)
        den_s = fg*den_g + (1-fg)*den_l
        N_rey = (den_l*Vm* (0.0254*d))/mhew_l
        f = (1/(-2*np.log10(2*e/(3.7*12*d)-(5.02/N_rey)* np.log10((2*e/(3.7*12*d)) +(13/N_rey)))))**2
        
        pres_grad = den_s * g * math.sin(theta) + (f*Vm**2*den_l*(1-fg))/(2*(0.0254*d))
        
    elif (den_g * Vsg**2)>test:
        print('Churn flow Exist')
        Co = 1.15
        fg = Vsg/(Co*Vm + V_tb)
        den_s = fg*den_g + (1-fg)*den_l
        N_rey = (den_l*Vm* (0.0254*d))/mhew_l
        f = (1/(-2*np.log10(2*e/(3.7*12*d)-(5.02/N_rey)* np.log10((2*e/(3.7*12*d)) +(13/N_rey)))))**2
        
        pres_grad = den_s * g * math.sin(theta) + (f*Vm**2*den_l*(1-fg))/(2*(0.0254*d))
        
elif Vsg > 3.1*(sur_t * g * (den_l-den_g)/den_g**2)**0.25:
    print('Annular flow Exists')
    V_crit = 10000*Vsg*mhew_g/sur_t * np.sqrt(den_g/den_l)
    if V_crit < 4:
        FE = 0.055* V_crit**2.86
    else:
        FE = 0.857*math.log10(V_crit) - 0.20
        
    if xg ==0:
        H_l = (FE*Vsl)/(FE*Vsl + Vsg)
    else:
        X = ((1-xg)/xg)**0.9 * np.sqrt(den_g/den_l)*(mhew_l/mhew_g)**0.1
        H_l = (1 + X**0.8)**(-0.378)
    
    Nreg= den_g * Vsg *(0.0254*d)/mhew_g
    Fc = 0.046*(1+75*H_l)* Nreg**(-0.2)
    den_c = (Vsg * den_g + Vsl * den_l * FE) / (Vsg + Vsl*FE)
    
    pres_grad = den_c * g * math.sin(theta) + (Fc * den_c)/(2*(0.0254*d)) * (Vsg/(1-H_l))**2
    
print('The estimated Pressure gradient in Pa/m is: {}' .format(pres_grad))

Churn flow Exist
The estimated Pressure gradient in Pa/m is: 5153.37794812138
